In [1]:
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import SimpleITK as sitk
from skimage import color
from skimage.filters import threshold_otsu, threshold_yen
from skimage.morphology import cube
import pandas as pd

plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (12, 10)

from subtle.subtle_preprocess import register_im, scale_im, zoom_iso
import subtle.utils.io as suio
from subtle.utils.experiment import get_experiment_data
from scipy.ndimage.morphology import binary_erosion, binary_dilation, binary_fill_holes
from skimage.measure import label as cc_label, regionprops
from ipywidgets import interact, interact_manual
from subtle.subtle_metrics import ssim

def get_rgb(img):
    img = (img - np.min(img))/np.ptp(img)
    return np.dstack((img, img, img))

def overlay_mask(data, label, r=0.3, g=0.1, b=0.1):
    data_rgb = get_rgb(data)

    label_r = label * r
    label_g = label * g
    label_b = label * b
    
    label_rgb = np.dstack((label_r, label_g, label_b))

    data_hsv = color.rgb2hsv(data_rgb)
    label_hsv = color.rgb2hsv(label_rgb)

    data_hsv[..., 0] = label_hsv[..., 0]
    data_hsv[..., 1] = label_hsv[..., 1] * 0.55

    return color.hsv2rgb(data_hsv)

/home/srivathsa/projects/SubtleGad/train/subtle/utils/hyperparameter.py:8: UserWarning: Module test_tube not found - hyperparameter related functions cannot be used
  warnings.warn('Module test_tube not found - hyperparameter related functions cannot be used')


<Figure size 432x288 with 0 Axes>

In [7]:
dirpath_gt = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data'
# dirpath_gt = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data'
dirpath_gad = '/home/srivathsa/projects/studies/gad/stanford/data/aa5daa_010c89'
# dirpath_gad = '/home/srivathsa/projects/studies/gad/tiantan/data/df6604_5f740b'
# dirpath_gad = '/home/srivathsa/projects/studies/gad/stanford/data/aa5daa_010c89'
case_num = 'Patient_0120'
ext = 'npy'
thresh_pcnt = 0.25
area_thresh = 500

all_data = suio.load_file('{}/{}.{}'.format(dirpath_gt, case_num, ext), params={'h5_key': 'all'}).astype(np.float32)
pc_mask = (all_data[1, :, 0] > 0)
pre_con = all_data[0, :, 0]
gt_full = all_data[0, :, 2]
gt_data = all_data[1, :, 2]
gt_data = gt_data / gt_data.mean()

In [8]:
gad_raw, _ = suio.dicom_files('{}/{}/{}_SubtleGad'.format(dirpath_gad, case_num, case_num))
gad_raw = gad_raw / gad_raw.mean()

In [9]:
pmap = sitk.GetDefaultParameterMap('affine')
gt_reg, _ = register_im(pre_con, gt_full, param_map=pmap)
gad_reg, _ = register_im(pre_con, gad_raw, param_map=pmap)

image registration
registration done, 17.4 s
image registration
registration done, 17.1 s


In [10]:
gad_data = pc_mask * gad_reg
gt_data = pc_mask * gt_data

gt_data = np.interp(gt_data, (gt_data.min(), gt_data.max()), (0, 1))
gad_data = np.interp(gad_data, (gad_data.min(), gad_data.max()), (0, 1))
pc_data = all_data[1, :, 0]
pc_data = pc_data / pc_data.mean()
pc_data = np.interp(pc_data, (pc_data.min(), pc_data.max()), (0, 1))
pc_data = scale_im(gt_data, pc_data)
gad_data = scale_im(gt_data, gad_data)

diff_gt = np.abs(gt_data - pc_data)
diff_gt = diff_gt / diff_gt.mean()
diff_gt = np.interp(diff_gt, (diff_gt.min(), diff_gt.max()), (0, 1))

diff_gad = np.abs(gad_data - pc_data)
diff_gad = diff_gad / diff_gad.mean()
diff_gad = np.interp(diff_gad, (diff_gad.min(), diff_gad.max()), (0, 1))

diff = np.abs(diff_gt - diff_gad)
sl = 119
# plt.imshow(np.hstack([pc_data[sl], gt_data[sl], gad_data[sl]]))
# plt.imshow(np.hstack([diff_gt[sl], diff_gad[sl]]))
# plt.imshow(diff[sl])

image intensity normalization
scaling done, 3.46 s
image intensity normalization
scaling done, 3.32 s


In [11]:
mask_er = binary_erosion(pc_mask, iterations=5)
diff = diff * mask_er

# th = threshold_yen(diff, nbins=16)
th = diff.max() * 0.25
dmask = (diff >= th)

# dmask = binary_erosion(dmask)
dmask = binary_dilation(dmask)
dmask = binary_fill_holes(dmask)
dmask = dmask.astype(np.uint8)

comps = cc_label(dmask)
rprops = regionprops(comps)

c_areas = [(r.label, r.area) for r in rprops]
reg_filter = [c for c in c_areas if c[1] >= area_thresh]
reg_filter = sorted(reg_filter, key=lambda c: c[1])[::-1]

filt_masks = np.array([(comps == m) for m, _ in reg_filter])
mask_comb = np.sum(filt_masks, axis=0).astype(np.uint8)
nz_slices = np.sum(mask_comb, axis=(1, 2))
disp_slices = [idx for (idx, sl_sum) in enumerate(nz_slices) if sl_sum >= 200]

disp_imgs = []

for sl_idx in disp_slices:
    gt_rgb = overlay_mask(gt_data[sl_idx], mask_comb[sl_idx])
    gad_rgb = overlay_mask(gad_data[sl_idx], mask_comb[sl_idx])
    disp_imgs.append((sl_idx, np.hstack([gt_rgb, gad_rgb])))

@interact
def show_imgs(idx=(0, len(disp_imgs)-1, 1)):
    plt.imshow(disp_imgs[idx][1])
    plt.title('Index = {}'.format(disp_imgs[idx][0]))
    plt.axis('off')

interactive(children=(IntSlider(value=123, description='idx', max=246), Output()), _dom_classes=('widget-inter…

In [25]:
bpath = '/home/srivathsa/projects/studies/gad/gen_siemens/data/Patient_0091'
pre_con, pc_hdr = suio.dicom_files('{}/14_Ax_T1_MPRAGE'.format(bpath))
low_con, _ = suio.dicom_files('{}/21_Ax_T1_MPRAGE_+C'.format(bpath))
full_con, fc_hdr = suio.dicom_files('{}/29_Ax_T1_MPRAGE_+C'.format(bpath))

pre_zoom, _ = zoom_iso(pre_con, np.array([1.0, 0.9375, 0.9375]), np.array([1.0, 0.5, 0.5]))
pre_zoom = np.pad(pre_zoom, pad_width=[(0, 0), (16, 16), (38, 39)], mode='constant', constant_values=0)

full_zoom, _ = zoom_iso(full_con, np.array([1.0, 0.9375, 0.9375]), np.array([1.0, 0.5, 0.5]))
full_zoom = np.pad(full_zoom, pad_width=[(0, 0), (16, 16), (38, 39)], mode='constant', constant_values=0)

In [26]:
dpath_ref = '/home/srivathsa/projects/studies/gad/gen_siemens/data/Patient_0091/21_Ax_T1_MPRAGE_+C'

suio.write_dicoms(dpath_ref, pre_zoom, 
                  '/home/srivathsa/projects/studies/gad/gen_siemens/data/Patient_0091/14_Ax_T1_MPRAGE_RS',
                  series_desc_pre='', series_num=14)

In [27]:
suio.write_dicoms(dpath_ref, full_zoom,
                 '/home/srivathsa/projects/studies/gad/gen_siemens/data/Patient_0091/29_Ax_T1_MPRAGE_+C_RS',
                 series_desc_pre='', series_num=29)

In [36]:
meta = suio.load_h5_metadata('/home/srivathsa/projects/studies/gad/gen_siemens/preprocess/data/Patient_0539_meta.h5')
meta['old_spacing_zero'] = meta['pixel_spacing_zero']
meta['old_spacing_low'] = meta['pixel_spacing_low']
meta['old_spacing_full'] = meta['pixel_spacing_full']

In [37]:
suio.save_meta_h5('/home/srivathsa/projects/studies/gad/gen_siemens/preprocess/data/Patient_0539_meta.h5', meta)

0

In [51]:
df_inf = pd.read_csv('/home/srivathsa/projects/studies/gad/gad_inf_repo.csv')
df_filt = df_inf[df_inf['Read?'] == 'No']

In [52]:
local_cases = [d['Case'] for d in df_filt.to_dict(orient='records') if 'LOCAL' in d['Inference_Path']]

In [53]:
print(len(local_cases))

46
